<a href="https://colab.research.google.com/github/CaeltUNIFEI/Infografico/blob/main/tratamentoDadosUNIFEI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
pd.set_option('display.max_columns', 500)

quantidadeLinhasExcluidas = 0

In [ ]:
dados = pd.read_csv('/content/drive/MyDrive/censoUNIFEI/Censo UNIFEI.csv', sep=',', encoding='utf-8')

# Extração e limpeza dos dados

In [ ]:
dados1 = dados.drop(columns=['Carimbo de data/hora', 'Termo de Consentimento e Livre Esclarecimento:'], )
dados1 = dados1.rename(columns={'Data de nascimento:' : 'dataNascimento', 'Sexo:' : 'sexo', 'Orientação sexual:' : 'orientacaoSexual', 'Etinia ou cor:' : 'etnia/cor', 'Religião:' : 'religiao', 'Município de nascimento:' : 'municipioNascimento', 'Unidade Federativa: ' : 'UF', 'Curso:' : 'curso', 'Data de ingresso:' : 'dataIngresso', 'Cursou o ensino médio em:' : 'EM', 'Na forma de ingresso, você utilizou qual categoria?' : 'categoriaIngresso', 'Caso tenha utilizado a cota, qual grupo você selecionou?' : 'grupoCota', 'Já trancou o curso?' : 'trancouCurso', 'Mora em República?' : 'republica', 'Você possui acesso a serviços básicos como água encanada, energia elétrica e coleta de lixo?' : 'servicosBasicos', 'Você trabalha?' : 'trabalha', 'se você trabalha, você tem acesso a benefícios como plano de saúde ou vale-refeição no seu trabalho?' : 'beneficiosTrabalho', 'Qual é a renda total mensal de sua família, considerando todas as fontes de renda, incluindo salários, aposentadorias, pensões, benefícios sociais, entre outros?' : 'rendaMensal', 'Quantas pessoas contribuem para a renda da sua família?' : 'contribuintes', 'Você ou sua família possuem dívidas ou empréstimos? ' : 'dividas/emprestimos', 'Recebe auxílio da UNIFEI?' : 'recebeAuxilio', 'Se recebe auxílio da UNIFEI, qual é?' : 'qualAuxilio', 'O que acha do preço do Restaurante Universitário da UNIFEI?' : 'precoRU', 'O que acha do suporte socioeconômico da UNIFEI?' : 'suporteSocioeconomico', 'Justifique a resposta anterior' : 'justifique', 'Utiliza de assistência da DAE?' : 'assistenciaDAE', 'Possui alguma necessidade física ou psicológica?' : 'possuiNecessidade', 'Se possui alguma necessidade física ou psicológica, qual(is) é(são)?' : 'qualNecessidade'})


In [ ]:
dados1['sexo'] = dados1['sexo'].replace({'Femenino' : 'Feminino'})

# Transformação dos dados

Idade (em anos)

In [ ]:
idadeArray = []

for data in dados1['dataNascimento']:
  idade = int(((date.today() - datetime.strptime(data, '%Y-%m-%d').date()).days)/365)
  idadeArray.append(idade)

dados1.insert(loc= 1, column= 'idade', value=idadeArray)


Data de ingresso

In [ ]:
tempoArrayAnos = []

for data in dados1['dataIngresso']:
  tempo = (date.today() - datetime.strptime(data, '%Y-%m-%d').date())
  tempoAnos = round(tempo.days / 365, 2)

  tempoAnos = np.around(tempoAnos / 0.5) * 0.5
  tempoArrayAnos.append(tempoAnos)

dados1.insert(loc= 10, column= 'tempoNaUnifeiEmAnos', value=tempoArrayAnos)

In [ ]:
print(f"Linha(s) removida(s): {dados1.query('idade < 18').shape[0]}")
dados1.drop(dados1[dados1['idade'] < 18].index, inplace=True) #Remove a linha onde a idade é menor que 18 anos

#print(f"Linha(s) removida(s): {dados1.query('tempoNaUnifeiEmAnos > 15').shape[0]}")
#dados1.drop(dados1[dados1['tempoNaUnifeiEmAnos'] > 15].index, inplace=True)

Linha(s) removida(s): 1
Linha(s) removida(s): 1


# Contagem dos dados

In [ ]:
contagemIdade = dados1['idade'].value_counts().sort_index()
contagemSexo = dados1['sexo'].value_counts()
contagemOrientacao = dados1['orientacaoSexual'].value_counts()
contagemEtnia = dados1['etnia/cor'].value_counts()
contagemUF = dados1['UF'].value_counts()
contagemTempoUnifei = dados1['tempoNaUnifeiEmAnos'].value_counts().sort_index()

In [ ]:
respostas = dados1['religiao']

mapeamento = {
    "ateísta": "Ateu", "ateu": "Ateu",
    "agnóstico": "Agnóstico",
    "católico": "Católico", "catolica": "Católico", "catolicismo": "Católico", "católica": "Católico", "catolico" : "Católico",
    "evangélico": "Evangélico", "evangélica": "Evangélico", "cristão protestante": "Evangélico", "cristã/evangélica": "Evangélico",
    "espiritismo": "Espiritismo",
    "cristão" : "Cristão",
    "nenhuma": "Sem religião", "não aplica": "Sem religião", "não possuo": "Sem religião", "não possuo religião.": "Sem religião", "nao tenho": "Sem religião", "desvinculado": "Sem religião", "não tenho" : "Sem religião", "não possuo religião" : "Sem religião", "sem religião" : "Sem religião", "não sei" : "Sem religião",
}

def normalizar_respostas(resposta):
    resposta = resposta.strip().lower()
    return mapeamento.get(resposta, '[ERRO]')

dados1['religiao'] = dados1['religiao'].apply(normalizar_respostas)
contagemReligiao = dados1['religiao'].value_counts()

In [ ]:
print("Linha(s) removida(s): " + str(dados1.query('religiao == "Cristão"').shape[0]))
dados1.drop(dados1[dados1['religiao'] == 'Cristão'].index, inplace=True)

Linha(s) removida(s): 2


In [ ]:
dados1.to_csv('/content/drive/MyDrive/censoUNIFEI/dadosLimpos.csv', sep=',', index=False, encoding='utf-8', decimal='.')

quantidadeLinhasExcluidas = dados.shape[0] - dados1.shape[0]

#Nao precisa adicionar +1 por causa do indice 0, ele retorna corretamente.
print(f"Quantidades de \"Entrevistados\": {dados.shape[0]}")
print(f"Quantidades de linhas excluidas: {quantidadeLinhasExcluidas}")
print(dados1.shape[0])